# Utilities for Thesis
### Figure generators and such

# Generic Imports

In [ ]:
import numpy as np
import madmom
import os
from scipy.io import wavfile

# Initialization

In [ ]:
# Load auido file

file_name = 'zztop_badtothebone_mono_5sec.wav'
audio_path_relative = '../../audio_for_thesis'

audio_path = os.path.join(os.getcwd(), audio_path_relative)
audio_path_absolute = os.path.abspath(audio_path)
if not os.path.exists(audio_path_absolute):
    raise Exception("Audio file path doesnt exist:", audio_path_absolute)
    
audio_path_full = os.path.join(audio_path_absolute, file_name)
if not os.path.exists(audio_path_full):
    raise Exception("Audio file doesnt exist:", audio_path_full)

fs, audio_buffer = wavfile.read(audio_path_full)

print(audio_path_full)
print('audio buffer shape:', audio_buffer.shape, 'sampling rate:', fs)

# Figures

### Time Domain Audio Signal Representation

### Frequency Domain Audio Signal Representation

### Time/Frequency Domain Audio Signal Representation

### Mel Scale

### Decibel Scale